In [1]:
import pandas as pd
import modulo as md
import requests
import time 
from bs4 import BeautifulSoup


In [2]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

Estos son los equipos ingleses que estan en primera y segunda division en la temporada el año actual 2023-2024 sin embargo a los equipos que hayan estado en 2 divisiones en los ultimos años se tomaran como equipos de la menor division en la que hayan estado por ejemplo el Luton equipo recien ascendido a premier league aunque actualmente este en primera division en general será tratado como equipo de segunda division.

In [3]:
equipos_ingleses=pd.read_csv('dataset\England_teams.csv')

In [4]:
temporadas=md.listnEditions(5,2023)


In [5]:
def get_teams_mean(nivelliga,equipos,anios):
#creo una lista donde voy a guardar 1 dataframe por equipo con el rendimiento de ese equipo durante las ultimas 5 temporadas
 
    resultados=[]

    '''defino una variable que me dice de en que division  deberian estar todos los equipos durante las temporadas,
    a los equipos que esten en segunda division o hayan estado en segunda durante los ultimos 5 años solo les estudiare 3 años de su historial
    y si durante esos 3 años estuvo en tercera division solo se le valeran los años que estuvo en primera o segunda.'''

    nivelliga='1'

    equipos_descendidos=[]
    #recorro el dataframe que guarda todos los equipos 
    for index, row in equipos.iterrows():
        #en el principio asumo que el equipo no ha descendido en las ultimas 5 temporadas
        descendio=False
        # elijo el equipo
        nombre=row['Nombre']
        codigoE=row['id Web']
        anios_en_nivel=0

        # creo una lista donde voy guardar los resulados del equipo en cada temporada
        temporadas_equipo=[]

        #recorro las temporadas obteniendo los datos del equipo esa temporada
        for temporada in temporadas:
            #muestro el equipo y la temporada que estoy scrapeando
            print(nombre,temporada, end='\r')
            url=f'https://fbref.com/es/equipos/{codigoE}/{temporada}/all_comps/'
            res = requests.get(url, headers=headers)
            soup=BeautifulSoup(res.text, 'html.parser')

            #si el equipo en algun momento estuvo de otra division diferente a la 1 creo un diccionario con sus datos y lo agrego a una lista
            # de diccionarios de equipos descendidos
            nivelactual=md.league_level(soup)
            if nivelactual != nivelliga and anios_en_nivel < anios:

                print('El ' + nombre+' estuvo en segunda en la temporada '+temporada + ' al momento van '+str(len(resultados))+ ' equipos', end='\n\r')
                equipo_descendido={}
                equipo_descendido['Nombre']=nombre
                equipo_descendido['id Web']=codigoE
                equipo_descendido['id_ml']=row['id_ml']
                equipos_descendidos.append(equipo_descendido)
                #activo descendio
                descendio=True
                #me salgo del bucle de temporadas
                
                break 

            #obtengo los resultados para el equipo en la temporada
            equipo=md.get_players_mean(res,url)
            #añado una columna llamada temporada donde guardare de que temporada es el registro
            equipo['temporada']=temporada
            #añado los resultados a la lista
            temporadas_equipo.append(equipo)
            anios_en_nivel+=1

            

        #si el equipo no descendio 
        if not descendio:
            
            #apilo todos los registros del equipo en un solo dataframe 
            resultado = pd.concat(temporadas_equipo, ignore_index=True)
            resultados.append(resultado)

    equipos = pd.concat(resultados, ignore_index=True)

    return [equipos, equipos_descendidos]
    
        



El Bournemouth estuvo en segunda en la temporada 2021-2022 al momento van 2 equipos
El Brentford estuvo en segunda en la temporada 2020-2021 al momento van 2 equipos
El Burnley estuvo en segunda en la temporada 2022-2023 al momento van 3 equipos


KeyboardInterrupt: 

In [23]:
url='https://fbref.com/es/equipos/ee7c297c/2020-2021/all_comps/Estadisticas-de-Cadiz-Todas-las-competencias'
res = requests.get(url, headers=headers)
soup=BeautifulSoup(res.text, 'html.parser')

In [24]:

# considero en cuantas competiciones participa el equipo para luego poder elegir las tablas 

tamaño=soup.find('div', class_='filter switcher')

if  tamaño is None:
     print('hola')


<div class="filter switcher" data-controls="#switcher_stats_standard">
<div class="current">
<a class="sr_preset" data-hide="#all_stats_standard .section_heading, #all_stats_standard .topscroll_div" data-show=".assoc_stats_standard_combined">Todas las competencias</a>
</div><div class="">
<a class="sr_preset" data-hide="#all_stats_standard .section_heading, #all_stats_standard .topscroll_div" data-show=".assoc_stats_standard_12">La Liga</a>
</div><div class="">
<a class="sr_preset" data-hide="#all_stats_standard .section_heading, #all_stats_standard .topscroll_div" data-show=".assoc_stats_standard_569">Copa del Rey</a>
</div></div>

In [11]:
if equipo[1] != '1':
    print('si')